# Kugelflächenfunktionen

## Vorbereitung

Führen sie zuerst die folgende Zelle aus, um die Simulation vorzubereiten. Sie können diese danach mit dem Pfeil links einklappen. 

In [ ]:
import numpy as np
import math
import matplotlib
import plotly.graph_objs as go
from plotly.subplots import make_subplots

### Creating a colormap that shows a black line at zero values ###
turbo_cmap = matplotlib.colormaps['turbo']
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        if k*h == 0.5:
            pl_colorscale.append([k*h, 'rgb'+str((0, 0, 0))]) # Farbe des Nulldurchganges
        else:
            pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

turbo = matplotlib_to_plotly(turbo_cmap, 255)
##################################################################

class spherical_harmonics:
    def __init__(self, grid_size=100):
        theta = np.linspace(0, np.pi, grid_size)
        phi = np.linspace(0, 2*np.pi, grid_size)
        self.theta_grid, self.phi_grid= np.meshgrid(theta, phi)
        
        self.x_grid = np.sin(self.theta_grid) * np.cos(self.phi_grid)
        self.y_grid = np.sin(self.theta_grid) * np.sin(self.phi_grid)
        self.z_grid = np.cos(self.theta_grid)
        
    def N_lm(self, l, m):
        return np.sqrt( 2 * (l + 1) * math.factorial(l-m) / ( 2 * math.factorial(l+m) ) )
    
    def P_lm(self, theta, l, m): # Achtung, theta, nicht cos(theta) wie normalerweise definiert
        if l==0: return 1
        elif l==1:
            if m==0: return np.cos(theta)
            elif m==1: return -np.sin(theta)
            elif m==-1: return np.sin(theta)/2
        elif l==2:
            if m==0: return (3*np.cos(theta)**2-1) / 2
            elif m==1: return -3*np.sin(theta)*np.cos(theta)
            elif m==-1: return np.sin(theta)*np.cos(theta) / 2
            elif m==2: return 3*np.sin(theta)**2
            elif m==-2: return np.sin(theta)**2 / 8
        elif l==3:
            if m==0: return (5*np.cos(theta)**3-3*np.cos(theta)) / 2
            elif m==1: return -3 * (5*np.cos(theta)**2-1) * np.sin(theta) / 2
            elif m==-1: return (5*np.cos(theta)**2-1) * np.sin(theta) / 8
            elif m==2: return 15 * np.cos(theta) * np.sin(theta)**2
            elif m==-2: return np.cos(theta) * np.sin(theta)**2 / 8
            elif m==3: return -15 * np.sin(theta)**3
            elif m==-3: return np.sin(theta)**3 / 48
    
    def Y_lm(self, theta, phi, l, m):
        Y_lm = 1/np.sqrt(2 * np.pi) * self.N_lm(l, m) * self.P_lm(theta, l, m) * np.exp(1.0j * m * phi)
        return Y_lm
    
    def plot_shape(self, l, m):
        assert l <= 3 and l >= 0 , 'Darstellung nur für l von 0 bis 3 möglich.'
        assert m >=0 and m <= l , 'm muss nichtnegativ und kleiner oder gleich l sein.' 
        
        spherical_shape = np.real(self.Y_lm(self.theta_grid, self.phi_grid, l, m))
        
        spherical_shape_positive = np.where(spherical_shape>0, spherical_shape, 0)
        x_shape_positive = self.x_grid * spherical_shape_positive
        y_shape_positive = self.y_grid * spherical_shape_positive
        z_shape_positive = self.z_grid * spherical_shape_positive
        
        spherical_shape_negative = np.where(spherical_shape<0, -spherical_shape, 0)
        x_shape_negative = self.x_grid * spherical_shape_negative
        y_shape_negative = self.y_grid * spherical_shape_negative
        z_shape_negative = self.z_grid * spherical_shape_negative
        
        surface_positive = go.Surface(x=x_shape_positive, y=y_shape_positive, z=z_shape_positive,
                             colorscale=[[0, 'rgb(0.63082,0.06868,0.00401)'], [1, 'rgb(0.63082,0.06868,0.00401)']])
        surface_negative = go.Surface(x=x_shape_negative, y=y_shape_negative, z=z_shape_negative,
                             colorscale=[[0, 'rgb(0.23915,0.20833,0.54686)'], [1, 'rgb(0.23915,0.20833,0.54686)']]) 
        
        fig = go.Figure(data = [surface_positive, surface_negative])
        fig.update_traces(showscale=False)
        fig.show()
        
        
    def plot_unit_sphere(self, l, m):
        assert l <= 3 and l >= 0 , 'Darstellung nur für l von 0 bis 3 möglich.'
        assert m >=0 and m <= l , 'm muss nichtnegativ und kleiner oder gleich l sein.' 
        
        spherical_shape = np.real(self.Y_lm(self.theta_grid, self.phi_grid, l, m))
        
        surface = go.Surface(x=self.x_grid, y=self.y_grid, z=self.z_grid, surfacecolor=spherical_shape, colorscale='Turbo')
        fig = go.Figure(data = [surface])
        fig.show()
        
    def plot(self, l, m):
        assert l <= 3 and l >= 0 , 'Darstellung nur für l von 0 bis 3 möglich.'
        assert m >=0 and m <= l , 'm muss nichtnegativ und kleiner oder gleich l sein.' 
        
        spherical_shape = np.real(self.Y_lm(self.theta_grid, self.phi_grid, l, m))
        spherical_shape_positive = np.where(spherical_shape>0, spherical_shape, 0)
        x_shape_positive = self.x_grid * spherical_shape_positive
        y_shape_positive = self.y_grid * spherical_shape_positive
        z_shape_positive = self.z_grid * spherical_shape_positive
        
        spherical_shape_negative = np.where(spherical_shape<0, -spherical_shape, 0)
        x_shape_negative = self.x_grid * spherical_shape_negative
        y_shape_negative = self.y_grid * spherical_shape_negative
        z_shape_negative = self.z_grid * spherical_shape_negative
        
        fig = make_subplots(rows=1, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}]],
                    subplot_titles=['Darstellung als Entfernung vom Ursprung', 'Darstellung auf der Einheitskugel'],
                    )
        
        surface_positive = go.Surface(x=x_shape_positive, y=y_shape_positive, z=z_shape_positive,
                             colorscale=[[0, 'rgb(0.63082,0.06868,0.00401)'], [1, 'rgb(0.63082,0.06868,0.00401)']])
        surface_negative = go.Surface(x=x_shape_negative, y=y_shape_negative, z=z_shape_negative,
                             colorscale=[[0, 'rgb(0.23915,0.20833,0.54686)'], [1, 'rgb(0.23915,0.20833,0.54686)']]) 
        
        fig.add_trace(surface_positive, 1, 1)
        fig.add_trace(surface_negative, 1, 1)
        fig.update_traces(showscale=False)
        
        surface_sphere = go.Surface(x=self.x_grid, y=self.y_grid, z=self.z_grid, 
                                    surfacecolor=spherical_shape, colorscale=turbo)
        fig.add_trace(surface_sphere, 1, 2)
        
        
        fig.show()

## Plotten

Durch das ausführen der nächsten Zeile starten sie die Simulation. Um verschiedene Zustände zu betrachten, ändern sie die Werte für l und m, wobei nur Zustände bis maximal l=3 dargestellt werden können.

In [ ]:
wasserstoff = spherical_harmonics()
wasserstoff.plot(l=3,m=2)